In [39]:
import trimesh
import os.path

file_path_1 = "/Users/diegotrap/Desktop/otto_body.stl"
file_path_2 = "/Users/diegotrap/Desktop/CABEZA 1.stl" #No watertigth
file_path_3 = "/Users/diegotrap/Desktop/otto_legs.stl" #Several parts
file_path_4 = "/Users/diegotrap/Desktop" #directory

## input
mesh = trimesh.load(file_path_3)
unit = "mm" #mm, cm, in

volume = mesh.volume / 1000
surface = mesh.area / 100
is_watertight = mesh.is_watertight
bounding_box_size = mesh.bounding_box.extents
number_of_bodies = mesh.split().size
    

print("volume = {:.2f}cm3".format(volume))
print("surface = {:.2f}cm2".format(surface))
print("is_watertight = {}".format(is_watertight))
print("bounding_box_size = {}mm".format(bounding_box_size))
print("number_of_bodies = {}".format(number_of_bodies))

#TODO meter trimesh.load en excepción para que si hay algún error nos lo notifique
#TODO generar imagen thumbnail
#TODO decidir donde van los controles de tamaño, watertight, volumen...
#TODO decidir si debemos dar todos los resultados en la unidad original del archivo y después tratar las unidades fuera
#TODO no sé como todo esto se mete en un json


volume = 12.54cm3
surface = 131.51cm2
is_watertight = True
bounding_box_size = [40.         38.00009155 38.50000381]mm
number_of_bodies = 2


# Quotator classes

Sliced volume formula

$$V_\text{sliced} = St + (V-St) \rho_i $$

where

* $V_\text{sliced}$ is the approximate sliced (or printed) volume, `sliced_volume`.
* $S$ is the mesh surface, `surface`.
* $V$ is the mesh volume, `volume`.
* $t$ is the printing shell thickness (or perimeter). We use $t = 1.2\text{mm}$
* $\rho_i$ is the printing infill ratio, expressed from 0 to 1. We use $\rho_i = 0.2$


In [40]:
SHELL_THICKNESS = 0.12 #cm
INFILL_RATIO = 0.20

DEFAULT_PPCM3 = 0.40 #€
FIXED_PRICE_PER_PART = 1.0 #€
                    #ppcm3 refers to the sliced volume, not the mesh volume



class PartQuotation():
    
    def __init__(self, path, unit="mm"):
        
        self.unit = unit
        # Setting conversion factors to convert the file units to mm
        if unit == "mm":
            conversion_factor = 1
        elif unit == "cm":
            conversion_factor = 10
        elif unit == "in":
            conversion_factor = 25.4
        else:
            raise Exception("Cannot recognize the unit code argument provided.")
        
        # Store the part name
        self.name = os.path.basename(path)
        
        
        # Load the mesh in trimesh and extract its parameters
        mesh = trimesh.load(path)
        
        self.volume = mesh.volume*conversion_factor**3 / 1000 #unit: cm3
        self.surface = mesh.area*conversion_factor**2 / 100   #unit: cm2
        self.is_watertight = mesh.is_watertight
        self.bounding_box_size = mesh.bounding_box.extents*conversion_factor #unit: mm
        self.number_of_bodies = mesh.split().size
        
        self.sliced_volume = self.surface*SHELL_THICKNESS + (self.volume - self.surface*SHELL_THICKNESS)*INFILL_RATIO #cm3
        
    def calculate_price(self, price_per_cm3=DEFAULT_PPCM3):
        price = FIXED_PRICE_PER_PART + self.sliced_volume * price_per_cm3
        return price
        
class OrderQuotation:
    def add_part_quotation(self, part_quotation):
        pass

# Quotator function

In [47]:
def quotator(path, unit="mm", price_per_cm3=DEFAULT_PPCM3, debug=False):
    
    if debug:
        print("DEBUG INFO: unit = {}".format(unit))
    
    # Checking if the path provided is a file or a directory
    if os.path.isfile(path) :
        if debug:
            print("DEBUG INFO: `path` is file".format(unit))        
        
        quoted_part = PartQuotation(path, unit)
        name = quoted_part.name
        price = quoted_part.calculate_price(price_per_cm3)
        
        print("{} - {:.2f}€".format(name, price))
    
    # if the path is a directory, quote all the files inside
    else:
        if debug:
            print("DEBUG INFO: `path` is dir".format(unit))
            
        files = os.listdir(path)
        
        for f in files:
            
            file_name, extension = os.path.splitext(f)
            
            if extension == ".stl":
                
                # Rebuild the part path
                if path[-1] == "/":
                    part_path = path + f
                else:
                    part_path = path + "/" + f
                
                
                quoted_part = PartQuotation(part_path, unit)
                name = quoted_part.name
                price = quoted_part.calculate_price(price_per_cm3)
        
                print("{} - {:.2f}€".format(name, price))
        
    

quotator(file_path_4)

otto_legs.stl - 7.05€
otto_body.stl - 15.55€
